1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음.
        - 문서 길이(input)가 길어 답변 생성에 시간 소요
3. 임베딩(수치화)
    - 벡터 데이터베이스에 저장
4. 질문이 있으면 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
# 0. 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 #######################################
loader = Docx2txtLoader('law_1.docx') # --> 인스턴스 생성

text_splitter = RecursiveCharacterTextSplitter( # --> 분할 설정
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장  #########################

## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)


In [ ]:

## 2.3 벡터 데이터베이스에 저장(in memory)
## [방법1]
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )

In [3]:
%pip install langchain-pinecone

  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.18
    Uninstalling aiohttp-3.11.18:
      Successfully uninstalled aiohttp-3.11.18
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [7]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)
# pc = Pinecone(api_key="PINECONE_API_KEY")

In [8]:
from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
    index_name='law-index',
    embedding=embedding,
    documents=document_list,    
)

database

In [9]:

# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############

## 3.1 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########

## 4.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2 프롬프트 변수에 값 설정(format 함수)
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)
 
## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI()

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'전세사기피해자 금융지원은 전세사기피해자 및 해당 법에 따른 임차인의 긴박한 주거안정을 보호하기 위해 국가 및 지방자치단체가 필요한 자금을 융자하거나 그 밖에 필요한 지원을 할 수 있는 것을 말합니다. 국가는 우선변제를 받지 못한 전세사기피해자에게 주택도시기금에서 주택의 임대차에 필요한 자금을 융자할 수 있습니다. 또한, 금융회사 등은 전세사기피해자등의 대출에 대한 채무의 불이행 및 대위변제를 관리하기 위해 특정 조치를 할 수 있습니다.'